# 🦜🔗 LangChain agent BabyAGI

In [ ]:
%pip install -r requirements.txt

In [ ]:
import faiss
from typing import Dict, List, Optional, Any
from langchain import LLMChain, PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain_experimental.autonomous_agents import BabyAGI
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import SerpAPIWrapper

from dotenv import load_dotenv

load_dotenv()

Embeddings because we need to remember.

In [ ]:
embeddings = AzureOpenAIEmbeddings(azure_deployment="text-embedding-ada-002")

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings, index, InMemoryDocstore({}), {})

A todo chain to generate proper todos.

In [ ]:
todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. You have a clear focus, so you write precise todos, but not more than 4. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=AzureChatOpenAI(deployment_name="gpt-35-turbo",
                       api_version="2023-07-01-preview",
                       temperature=0), prompt=todo_prompt)

We need tools!

In [ ]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]

Setup the prompt for the agent

In [ ]:
prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

Start the agent.

In [ ]:
llm = AzureChatOpenAI(deployment_name="gpt-35-turbo",
                    api_version="2023-07-01-preview",
                    temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)
max_iterations: Optional[int] = 5
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=False,
    max_iterations=max_iterations,
)

baby_agi({"objective": "Write an euphoric blog article about the code.talks conference in hamburg."})

Feel free to send it on another mission :-)